<a href="https://colab.research.google.com/github/adelic-matf/MM/blob/main/Simbolicke.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Biblioteka SymPy

Sada ćemo se pozabaviti simboličkim izračunavanjem koje nam omogućava da rešavamo matematičke zadatke na računaru na način kako bismo ih rešavali na papiru.
Simboličko izračunavanje omogućava tačan zapis realnih brojeva,
pojednostavljivanje izraza, diferenciranje funkcija, izračunavanje graničnih
vrednosti, kao i rešavanje algebarskih i diferencijalnih jednačina.


Za sve to se koristi biblioteka SymPy. Detaljne informacije se mogu pronaći na linku [SymPy dokumentacija](https://docs.sympy.org/latest/tutorials/intro-tutorial/index.html#intro-tutorial).

Učitajmo biblioteku pod imenom sym.

In [1]:
import sympy as sym

Brojevi tipa float u Python-u (kao i u većini programskih jezika) predstavljeni su u binarnoj formi, koristeći standard IEEE-754. Razlomak 1/10 nema konačan binarni zapis, pa se zato čuva samo kao aproksimacija, tj. zaokružena vrednost.

Ovo lako možemo videtu u sledećem primeru.

In [2]:
a_comp = 1/10 + 1/5 #0.1 + 0.2
print(f"{a_comp:.17f}")
print(a_comp - 0.3)

0.30000000000000004
5.551115123125783e-17


Dakle, zbog zaokruživanja razlika više nije jednaka nuli, već se pojavljuje mala računska greška.

Međutim, racionalne brojeve možemo predstaviti i tačno ukoliko koristimo klasu  Rational iz biblioteke SymPy.

In [3]:
a_sym = sym.Rational(1, 10) + sym.Rational(1,5)
print(type(a_sym))
print(a_sym - 0.3)


<class 'sympy.core.numbers.Rational'>
0


Pri tome treba imati na umu da se radi o simboličkim reprezentacijama brojeva; pri pojavi decimalnih brojeva u izrazu, SymPy uvodi sopstvenu numeričku reprezentaciju, ali izraz i dalje ostaje simbolički.

## Simboličke promenljive
Simboličku promenljivu u biblioteci SymPy definišemo pomoću metode Symbol

 x = sym.Symbol('x')

gde je:

'x' — naziv simboličke promenljive,

x — Python objekat koji predstavlja simboličku promenljivu.

In [4]:
x = sym.Symbol('x')

Više simboličkih promenljivih možemo definisati odjednom pomoću funkcije symbols:

In [5]:
y, z = sym.symbols('y z')

Nakon što smo definisali simboličke promenljive, možemo definisati matematičke izraze:

In [6]:
izraz=(x+y)**2

Možemo ih i izračunavati:

In [7]:
sym.expand(izraz)


x**2 + 2*x*y + y**2

Postoji i funkcija za pojednostavljivanje izaraza. Npr. sledeći izraz neće biti ispisan u najjednostavnijem obliku

In [8]:
izraz2 = izraz-2*x*y
print(izraz2)

-2*x*y + (x + y)**2


Ali, ako upotrebimo sunkciju **simplify** izraz će biti sređen:

In [9]:
sym.simplify(izraz2) #pojednostavi izraz


x**2 + y**2

S tim u vezi, prilikom definisanja simboličkih promenljivih, u SymPy-ju je moguće zadati pretpostavke o njihovim osobinama (npr. da li su realne, pozitivne, celobrojne i sl.).
Ove pretpostavke omogućavaju sistemu da izvrši dodatna pojednostavljenja izraza.
Opšti oblik je:
```python
x = sym.Symbol('x', assumption1=True, assumption2=True, ...)
```
Na primer:

In [27]:
x = sym.Symbol('x', real=True)

U simboličkim izrazima dostupno je korenovanje, stepenovanje, trigonometrijske, eksponencijalne, logaritamske i hiperboličke funkcije, tj. SymPy imam simboličke verzije svih elementarnih matematičkih funkcija. Čak i više od toga, Dirakova delta funkcija, Gamma funkcija, Erf, itd. Kompletan spisak se može pronaći [ovde](https://docs.sympy.org/latest/modules/functions/special.html).

In [10]:
izraz3=sym.sin(x)**2+sym.cos(x)**2
print(izraz3)

sin(x)**2 + cos(x)**2


In [11]:
sym.simplify(izraz3)

1

## Simboličke funkcije

Najčešći i najjednostavniji način za definisanje simboličkih izraza je direktnim korišćenjem simboličkih promenljivih, kao što smo radili do sada.

Važno je znati da u SymPy možemo definisati i nepoznatu funkciju. Za to koristimo klasu **Function**.
Konkretno, simboličku funkciju definišemo pomoću konstruktora Function, na primer:

imeprom = Function('f')

gde je:
- imeprom - ime promenljive kojom ćemo pristupati simboličkoj funkciji
- 'f' je ime simboličke funkcije - može biti bilo koji string.

In [12]:
fja=sym.Function('f')
print(fja)

f


Na ovaj način smo definisali  simboličku funkciju f koju možemo pozivati u izrazima, ali koja još nema definisanu formulu - nepoznata je. Ako imamo definisanu simboličku promenljivu x tada izraz

In [13]:
fja(x)

f(x)

predstavlja simboličku funkciju f(x).

## Diferenciranje izraza i funkcija

U biblioteci SymPy diferenciranje se vrši pomoću funkcije diff.
Na primer,
```python
diff(f, x)
```
računa prvi izvod funkcije/izraza f po promenljivoj x, dok

```python
diff(f, x, n)
```
daje n-ti izvod.

Na primer ako imam izraz

In [14]:
f1 = x**2 + sym.sin(x)

drugi izvod se dobija komandom

In [15]:
sym.diff(f1,x,2)

2 - sin(x)

ili ekvivalentno

In [16]:
f1.diff(x,2)

2 - sin(x)

Komanda sym.diff predstavlja funkciju, dok je f.diff metoda objekta f; obe predstavljaju različite interfejse ka istom internom mehanizmu diferenciranja.

Možemo differencirati i simboličke funkcije za koje nemamo definisan izraz. Ako diferenciramo takvu funkciju  $f$ dobijamo simbolički izraz $df/dx$, kako i treba.

In [17]:
f = sym.Function('f')(x)
f.diff(x,1)

Derivative(f(x), x)

## Rešavanje (diferencijalnih) jednačina

Pokažimo kako se biblioteka SymPy koristi za rešavanje algebarskih i  diferencijalnih jednačina. Iskoristimo sledeći primer za to.

**Primer1:** Pretpostavimo da je ispaljen mali projektil početnom brzinom $\vec{v}$, pod uglom $\alpha$  u odnosu na tlo, a sa visine $h$. Odrediti trajekotoriju projektila.
Izaberimo referentni sistem tako da je koordinatni početak na nivou tla, a osa
$x$ usmerena u pravcu horizontalne projekcije početne brzine. U tom izboru koordinata važi $\cos\alpha\geq 0$ i $0\leq \alpha\leq\pi/2$.
Koristeći drugi Njutnov zakon, dobijamo sistem diferencijalnih jednačina drugog reda
$$
m\ddot{\vec{r}} = - m g\, \vec{k},
$$
gde je $\vec{r}(t)=\vec{r}(t) = \bigl(x(t),\, y(t),\, z(t)\bigr),$ vektor položaja projektila, $g$ intenzitet gravitacionog ubrzanja, a $\vec{k}\bigl(0,\, 0,\,1\bigr)$ jedinični vektor usmeren vertikalno naviše. Tačka iznad simbola označava izvod po vremenu.


**Rešenje:** Prvo primetimo da treba da definišemo jednu simboličku promenjivu $t$, i tri simboličke funkcije $x$, $y$ i $z$ koje zavise od $t$. Takođe, koeficijent $g$ mora biti  definisan simbolički, jer uvođenje numeričkih konstanti tipa float može dovesti do problema pri simboličkom rešavanju.

In [18]:
t = sym.Symbol('t', real=True)
g = sym.Symbol('g', positive=True)

x = sym.Function('x')(t)
y = sym.Function('y')(t)
z = sym.Function('z')(t)

Sada je potrebno da definišemo jednačine.

U biblioteci SymPy, metoda Eq služi za konstruisanje simboličke jednačine.

Osnovni oblik je:
```python
sym.Eq(lhs, rhs)
```

gde su:

lhs — leva strana jednačine

rhs — desna strana jednačine

Rezultat nije logička vrednost, već simbolički objekat koji predstavlja relaciju jednakosti. Mi ćemo konstruisati tri jednačine, pri čemu koristimo funkciju diff za definisanje izvoda.

In [19]:
eqx = sym.Eq(sym.diff(x, t, 2), 0)
eqy = sym.Eq(sym.diff(y, t, 2), 0)
eqz = sym.Eq(sym.diff(z, t, 2), -g)

Argumenti lhs  i rhs su ujedno i atributi objekta, pa im možemo pristupiti sa obj.lhs i obj.rhs.

In [20]:
eqz.lhs

Derivative(z(t), (t, 2))

In [21]:
eqz.rhs

-g

Preostalo je da rešimo jednačinu. Napomenimo da se obične (algebarske) jednačine, koje ne sadrže izvode, u SymPy-ju se rešavaju pomoću funkcije **solve**, dok se diferencijalne jednačine rešavaju pomoću funkcije **dsolve**.

In [22]:
solx = sym.dsolve(eqx)
soly = sym.dsolve(eqy)
solz = sym.dsolve(eqz)

solx, soly, solz

(Eq(x(t), C1 + C2*t), Eq(y(t), C1 + C2*t), Eq(z(t), C1 + C2*t - g*t**2/2))

Ovim smo dobili opšte rešenje, tj. beskonačno mnogo njih.
Da bismo dobili jedinstveno rešenje, nephodni su početni uslovi. I teksta zadatka imam da je
$$
\vec{r}(0)=h\vec{k} \quad \text{i} \quad
\dot{\vec{r}}(0)=\vec{v}=|\vec{v}|\cos\alpha \vec{i}+|\vec{v}|\sin{\alpha}\vec{k}.
$$

Funkcija dsolve kao drugi argument prima početne ili granične uslove. Sintaksa je
```python
dsolve(jednacina, ics={uslov1, uslov2, ...})
```

Ostaje da razjasnimo kako se zadaju ovi uslovi. Svaki početni (ili granični) uslov definiše vrednost nepoznate funkcije u nekoj tački, odnosno vrednost funkcije ili njenog izvoda u toj tački.

Za izračunavanje vrednosti funkcije ili simboličkog izraza u nekoj tački, u biblioteci SymPy koristi se metoda **subs**.

Ako je $x(t)$ funkcija nezavisne promenljive $t$, tada se njena vrednost u tački $t=0$, kao i vrednost njenog izvoda u istoj tački, mogu izračunati pomoću:
```python
x.subs(t, 0)
sym.diff(x, t).subs(t, 0)

In [23]:
#definisemo simbolicke konstante
h = sym.Symbol('h', real=True)
v = sym.Symbol('v', positive=True)      # v = |vec{v}|
alpha = sym.Symbol('alpha', real=True)

# pocetni uslovi
icsx = {x.subs(t, 0): 0, sym.diff(x, t).subs(t, 0): v*sym.cos(alpha)}
icsy = {y.subs(t, 0): 0, sym.diff(y, t).subs(t, 0): 0}
icsz = {z.subs(t, 0): h, sym.diff(z, t).subs(t, 0): v*sym.sin(alpha)}

#resavamo jednacine
solx = sym.dsolve(eqx, ics=icsx)
soly = sym.dsolve(eqy, ics=icsy)
solz = sym.dsolve(eqz, ics=icsz)

# stampamo resenja
print(solx)
print(soly)
print(solz)

print("\nStampanje resenja u eksplicitnom obliku:\n")
print(f"{solx.lhs}={solx.rhs}")
print(f"{soly.lhs}={soly.rhs}")
print(f"{solz.lhs}={solz.rhs}")

Eq(x(t), t*v*cos(alpha))
Eq(y(t), 0)
Eq(z(t), -g*t**2/2 + h + t*v*sin(alpha))

Stampanje resenja u eksplicitnom obliku:

x(t)=t*v*cos(alpha)
y(t)=0
z(t)=-g*t**2/2 + h + t*v*sin(alpha)


Time smo dobili trajektoriju projektila
$$\vec{r}(t)=(x(t),y(t), z(t)).$$

Eliminacijom promenljive  $t$ možemo odrediti jednačinu putanje u
$xz$-ravni. Nakon što eliminišemo vreme, dobijamo algebarsku jednačinu, pa za njeno rešavanje koristimo funkciju **solve**.
Osnovni oblik poziva je:
```python
sym.solve(f, x)
```
gde je:

$f$ — algebarska jednačina ili izraz oblika $f(x)=0$,

$x$ — nepoznata promenljiva po kojoj se rešava jednačina $f(x)=0$.

Važno je napomenuti da funkcija solve uvek vraća **listu** rešenja!

Čak i kada jednačina ima jedno rešenje, rezultat je lista, pa se tom rešenju pristupa, na primer, pomoću:
```python
sym.solve(jednacina, promenljiva)[0]
```
Pređimo na implementaciju. Prvo definišimo simboličku pormenljivu $X$. Jednačina po $x$ je $solx.rhs=solx.lhs$, gde  je na desnoj strani funkcija $x(t)$ koju sada želimo zameniti sa simboličkom promenljivom $X$. Osim toga jednačinu moramo pripremiti za solve tako da je na desnoj strani jednakosti 0. Sve to je urađeno u sledećim linijama koda:

In [24]:
# definisemo nezavisnu promenljivu X (nije x(t)!)
X = sym.Symbol('X', real=True)

# izrazimo iz prve jednacine t: t = X / (v cos alpha)
t_expr = sym.solve(solx.rhs-X, t)[0]
print(f"t= {t_expr}\n")

# z kao funkcija od X
z_X = sym.simplify(solz.rhs.subs(t, t_expr))

print(z_X)

t= X/(v*cos(alpha))

-X**2*g/(2*v**2*cos(alpha)**2) + X*sin(2*alpha)/(2*cos(alpha)**2) + h


Odredimo gde će projektil pasti na zemlju, tj. rešenje jednačine $z(x)=0$.

In [25]:
x_padovi = sym.solve(z_X, X)
print(x_padovi)

[v*(v*sin(2*alpha) - 2*sqrt(2*g*h + v**2*sin(alpha)**2)*Abs(cos(alpha)))/(2*g), v*(v*sin(2*alpha) + 2*sqrt(2*g*h + v**2*sin(alpha)**2)*Abs(cos(alpha)))/(2*g)]


Jednačina je kvadratna, pa smo dobili dva rešenja. Kako je $(0\leq \alpha\leq \pi/2)$ to je $|\cos(\alpha)|=\cos(\alpha)$ i onda je prvo rešenje negativno, pa ne može predstavljati mesto gde je projektil pao. Dakle, rešenje koje tražimo je

In [26]:
x_pad=x_padovi[0]
print(x_pad)

v*(v*sin(2*alpha) - 2*sqrt(2*g*h + v**2*sin(alpha)**2)*Abs(cos(alpha)))/(2*g)
